# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_cities_data = pd.read_csv("Output_Data/cities.csv")
weather_cities_data


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [11]:
humidity = weather_cities_data["Humidity"].astype(float)
locations = weather_cities_data[["Lat", "Lng"]]

In [22]:
google_heat_map = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=40,
                               point_radius =1.5)
google_heat_map.add_layer(heat_layer)
google_heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#Create data frames by filtering to your criteria; created three data frames
perfect_temperate_df=weather_cities_data.loc[(weather_cities_data['Max Temp']>=60)&(weather_cities_data['Max Temp']<=80)]
wind_df=perfect_temperate_df.loc[(temp_file['Wind Speed']<=8)]
perfect_weather_city=wind_file.loc[(wind_file['Cloudiness']==0)]

perfect_weather_city.sort_values(by='Country', ascending=False)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
433,433,svodin,0,SK,1558378878,72,47.91,18.50,63.00,6.93
508,508,noumea,0,NC,1558378811,77,-22.28,166.46,64.40,2.24
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24
298,298,mikkeli,0,FI,1558378844,59,61.69,27.27,62.60,6.93
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
371,371,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

#Create for loop and parameters
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=3, sort_keys=True))
    
    #Include try to get rid of the errors
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipping Missing Field")
hotel_df

{
   "html_attributions": [],
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -18.0446824,
               "lng": 13.8335005
            },
            "viewport": {
               "northeast": {
                  "lat": -18.04348247010728,
                  "lng": 13.83472832989272
               },
               "southwest": {
                  "lat": -18.04618212989272,
                  "lng": 13.83202867010728
               }
            }
         },
         "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
         "icon_background_color": "#909CE1",
         "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
         "name": "Opuwo Country Lodge",
         "photos": [
            {
               "height": 2592,
               "html_attributions": [
                  "<a href=\"https://maps.google.com/maps/contri

{
   "html_attributions": [],
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -22.3906663,
               "lng": 18.6153764
            },
            "viewport": {
               "northeast": {
                  "lat": -22.38871372010728,
                  "lng": 18.61680377989272
               },
               "southwest": {
                  "lat": -22.39141337989272,
                  "lng": 18.61410412010728
               }
            }
         },
         "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
         "icon_background_color": "#909CE1",
         "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
         "name": "West Nest Lodge",
         "photos": [
            {
               "height": 3120,
               "html_attributions": [
                  "<a href=\"https://maps.google.com/maps/contrib/10

{
   "html_attributions": [],
   "next_page_token": "Aap_uED7eZieEuUM7hfELNVf79UajChgxc33PRUZhMiA9M7W4W2pfOmvIkhvDcXlDi6ucpCYiHKeMTo83qA3jegdkulWyF_9H4Z5sMkiFBIfVxwnGGF1W1tAAzBR5IR8jpnl8a_M2sZsDw8mDI202g4Ows8zOJTSdZROZjk3ZnU2wZ38Poe7YmmY0ToIGj5MyMTe7AcYIwsoGka3ySpmUOe_rGCf0j1P6iwuArtDdqvy5QQnLTh7ps62Lt4yEKVHJrPd3MP4nvzRC2Yz7ivZQn-1jr5D4EWwzu1uEiHYXy_gjCK70ncPaQySQJv0055MvwWm8EuuWfivdicybv8xxzv8hsBDiStfFVUjRp9PULL7LfbXzesD0KFxPMG2odSUj1fzwpvZoHZHT9J6CUGNDrRRZh2OuVDr3MGcNUdGn5vUXzFzlZV3K7RJyl4xx3TtrF8Z",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -20.6441728,
               "lng": -46.1058822
            },
            "viewport": {
               "northeast": {
                  "lat": -20.64274842010728,
                  "lng": -46.10453982010728
               },
               "southwest": {
                  "lat": -20.64544807989272,
                  "lng": -46.10723947989273
          

{
   "html_attributions": [],
   "next_page_token": "Aap_uECJTs2-NSMAfrPD-2U6fk5oXdlpdpos2ZT7IaRv4CxLdkSmEeseGylFTeL8Nky7ggArgeBZBU-UEd0QWfUGAF6HyFFBBDTp1Zao77oGcxnp30dSts9wvJ4Sm9EDVjexkAAlC6A9IAPMn4-akLUfKfqR0SJg1oyRz5hoW4kRKqgyZzSfq4tF2_BBr10uDoKQQSJ9QTPGKJAypWzAxiMXKzZ4Tbk5lxKe_kkzQOxApZwfFWmocnMNG5U9fMXWHauae8fqtXHr6bcmlKTr1Yx_NaeZJ-HTntWnFknCl4ruB4jQqO9cfTrEq4gSHzfTmQ7EOavTDot5COSEqDJCgBKBwSxhD3ynCdVpUD_ZX9JmcVNddqDmbcObrncImYKGhVOHVl_WhdZmoIsNVtlbGUUvriHQWaX2UDKfYXhat6KLvE1IvVVaVNRS_gYM0DKJVLDb",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -23.5418436,
               "lng": 148.1730014
            },
            "viewport": {
               "northeast": {
                  "lat": -23.54049587010728,
                  "lng": 148.1743681798927
               },
               "southwest": {
                  "lat": -23.54319552989272,
                  "lng": 148.1716685201073
            

{
   "html_attributions": [],
   "next_page_token": "Aap_uECe9AOClfVZuADvqGgD96m3Q9TZCRpYB75A5T7ZIDeTuZNlqCwBJ-nflVPosxkrCposTQEnZg3qbo0XlZNzN3qlWIQLF8EHaNeZwRDnl1kuMx02vNquspnigRdXWX1gqP0bdo9kk2MTFbCkQ0_70RNhhqFk3d3Nf4sa6GFKXDwV0vgc915UxuZWHYixded3PkF_Kn5xQzno3TrVl98pxf4xbtM7XX8XpwCSiIzbzA-ID6KiGP-md4fbQrdzQLRsGloTfmTOLyuHXNuqCbVBLdTYO6pKYi_T1O_bdUayixrsuqyT1svPIwF2NzDndmhEiTyOcsvvZVdgrkwwGmS-U3oHHSp9x1r0-yIRBXHBVd1ai4NiXl3Rz5YarsKdyK2Dr-MP0yK_81DGjilZetCsRw81_E7C97VdWvHQzRx9nTr8acSW86d_AKQ6ULCGMpr6",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -29.7616494,
               "lng": -50.0155193
            },
            "viewport": {
               "northeast": {
                  "lat": -29.76033302010728,
                  "lng": -50.01408392010728
               },
               "southwest": {
                  "lat": -29.76303267989272,
                  "lng": -50.01678357989272
          

...

{
   "html_attributions": [],
   "next_page_token": "Aap_uECA5ho7Cy4_PVw4wBXIqOXl00PybWh1XB5hvOIwvONbPWVD9xMdhH5ScQzNblCnzZpgWEozLbclQMQtr_B9ZV8hTEmYAe0veoIYFFkTPt81tjoKIDcPAepEiiApKcEszGzQ8tfTehXOz-alSSdEpykhanCsCxwjZWIT3IRAr7OR_g7cNEkx0dv9pdbrQUat22YZWe_yXsY-cug43y_W48kuviwYvDYiu3-8mC4XOTIcqrpctWMkM38gIQ79Wo7Pg7VXVB9oRI2dP90SxBC21Nwg8GIs-6d-V6uUsHyuoB1h1pqRzCAeDoVq8sBuRViHMbIgZOZYsPO0wWrVSqnKkyKpZSpBf3qLCE4Fp-mKFfMJk8wl6DupbD_j7npVDv21y9aRK_vGagP9ASY17ud0o_kkimzYfUPe8pWw4EkP-CvgQb1a3OvbPkNTbFyXSTK9",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -24.811201,
               "lng": 152.4615936
            },
            "viewport": {
               "northeast": {
                  "lat": -24.80979357010728,
                  "lng": 152.4630594798927
               },
               "southwest": {
                  "lat": -24.81249322989272,
                  "lng": 152.4603598201072
             

{
   "html_attributions": [],
   "next_page_token": "Aap_uEAReSlbkHneGbxde0QqtLYdjnZ_UXXG7VtsBP5_grtCqGGERY07jvtgTBMlXm22X-pjKjKSptXBOgTdmyQiuL6EY_8Y2RQz7fRAOQ-FRRRdJU26rK62GblON3kMWqEdjBeId4Wrp7hPr34xP28qKXGqR5Oa9OQzFLTgldTR_30EswSTZULaqYVsCK9hBu5EzaIjbxrgueKkb8Pg9SRcdiBtAKojsOhgD42MGPA9_kDWIgB_A5pMW8ScgU8uU8T0cF2FoiS--_4oJQXv1jQSIatzUAIFu-zUHgCp1lZssqaWg2tJDBSA0q-y7Ny57YVrG9mdVVXDqlrnH2eS1YY5Wm59EMwh-8nr4FMl7nqCpBk5tCIjozbv_nofB3yDp3MBshcfAx4k68ujEw__Z1d-4ZMuCo8FAbWnQVdMFqjeROOyTfByDJkJ1s4w2cTvqCYe",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -22.3335063,
               "lng": 166.4101131
            },
            "viewport": {
               "northeast": {
                  "lat": -22.33215647010728,
                  "lng": 166.4114629298927
               },
               "southwest": {
                  "lat": -22.33485612989272,
                  "lng": 166.4087632701072
            

{
   "html_attributions": [],
   "next_page_token": "Aap_uEBqYxRXAy1IxYd6bQ4NfnjebmR0Bb2eDaB7J4PmSKdLmWcrfOZvjDDYN-UneUifgGTNYrQ981TQfeJWgARIzomJZW08CDlmjpxHsTjRdrgAByO3Y5Z5A6cL8ucPRT0yg7QeGSE1N2kXQ43VSZx4FYMHPxRuvfSTCsVv7ed8zhIo_N_DNxjtLtf8Crc2cJnUNQBBDYvstl4VluY9HVSUKrGBynmqDQmDHFxd8dDlGn2O8YS0hN2VYRDksmdBgZInUZ9-fBJ_MXmzGetK_jVj6IU10LDfSlesf2Uya4MNOKhuEZFEnbGHD2nctAhFKIm5thnmNlhzMQ2qL31XXrshfnN-wkoGXGxZEfPUa6hWYs2sTuPbtg9uc3U1JMmDRqM2S0mk8_J6ehyJ06HuhOIXyVc3R3l52wRiEK51oC_37-UuTielJTtVmqAS2HHrDEM_",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": 61.15855000000001,
               "lng": 24.3213001
            },
            "viewport": {
               "northeast": {
                  "lat": 61.15993157989272,
                  "lng": 24.32271187989272
               },
               "southwest": {
                  "lat": 61.15723192010727,
                  "lng": 24.32001222010728
         

{
   "html_attributions": [],
   "next_page_token": "Aap_uEDJqPB9M2NIyM5cp5WfFKFm2-A96zbKTQa7LVq6EPk-kojOxeQ4WapJp5B_RHJvnBoLjEGS60b1QAH86qn09dReTzPY9ctAmNQe402ovtTcKn45_9NpjeVgJHElpkgnTuUG7HPd8MrdhduhEaZVt-jxB4Z7i0bB_Zy68lcF_CorT45-ZIDXCsvMzFUCml2wXI7wIJ_LjdoSRREpt2NTFOS1daS5YTbcOfUGbvSAxWPiQxDBbh6wIN43tpAVXzTD0fTbCCCX-euWsEheGH8NI5SiZ41-Gtenri2HD5t6qLVdfWIq6Ow9-khZiEioOwX6vnSHrTUoLPIUof_sYX2StbWiRQbBxu7DZvvvkNRH2Mcdoson6qVpwS2IlurxMhIbfk43ldZoMNnQXrjclDdMGVa9GMvLFP2c_k2iMlj3mIQ8t8V8mzz_tDpFWocD-4pK",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": 43.25001899999999,
               "lng": 5.783575
            },
            "viewport": {
               "northeast": {
                  "lat": 43.25151902989271,
                  "lng": 5.785079979892722
               },
               "southwest": {
                  "lat": 43.24881937010727,
                  "lng": 5.782380320107278
           

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,West Nest Lodge
2,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Balneário do Lago Hotel
3,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,EMERALD EXPLORERS INN
4,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Capão da Canoa Mar Hotel
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,The LaLiT Golf & Spa Resort Goa
6,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24,The Temple House
7,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
8,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,Hedland Hotel
9,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,Isa Hotel


In [32]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [35]:
markers = gmaps.marker_layer(hotel_locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

google_heat_map.add_layer(markers)
google_heat_map


Figure(layout=FigureLayout(height='420px'))